In [19]:
##### DEFINE PRODUCT TO BE PLOTTED AND SET UP THE VARIABLES & NAMES

import numpy as np
import os
import fnmatch
import matplotlib as plt
import matplotlib
import matplotlib.pylab as plt
from matplotlib.transforms import Bbox
from skimage import exposure, transform
import glymur

#prefix = '/eodata/Sentinel-2/MSI/L1C/2017/02/10/S2A_MSIL1C_20170210T100131_N0204_R122_T32TQM_20170210T100132.SAFE/'
#prefix = '/eodata/Sentinel-2/MSI/L1C/2017/02/10/S2A_MSIL1C_20170210T100131_N0204_R122_T33TUG_20170210T100132.SAFE/'
#prefix = '/eodata/Sentinel-2/MSI/L1C/2017/03/11/S2A_MSIL1C_20170311T103011_N0204_R108_T32TMT_20170311T103014.SAFE/'
#prefix = '/eodata/Sentinel-2/MSI/L1C/2017/01/01/S2A_MSIL1C_20170101T100412_N0204_R122_T33TUF_20170101T100407.SAFE/'
prefix = '/eodata/Sentinel-2/MSI/L1C/2017/04/02/S2A_MSIL1C_20170402T124701_N0204_R138_T33XWH_20170402T124816.SAFE/'
# L2A product, fail: prefix = '/data/T31TEG/S2A_USER_PRD_MSIL2A_R008_V20160826T104022/'

prod_name = prefix[-66:-6]
prod_date = prefix[-55:-47]

colors = [None]*3

print('get all the required names/strings of the product... ')

for item in os.listdir(prefix):
    if os.path.isdir(os.path.join(prefix, item)):
        if item == 'GRANULE':
            granule = os.path.join(prefix,item)
            for item1 in os.listdir(granule):
                subgranule = os.path.join(granule,item1)
                tileID = subgranule[-30:-24]
                for item2 in os.listdir(subgranule):
                    if item2 == 'IMG_DATA':
                        img_data = os.path.join(subgranule, item2+'/')
                        for item3 in os.listdir(img_data):
                            
                            if fnmatch.fnmatch(item3, '*B04*'):
                                colors[2] = item3
                            if fnmatch.fnmatch(item3, '*B03*'):
                                colors[1] = item3
                            if fnmatch.fnmatch(item3, '*B02*'):
                                colors[0] = item3

name_out = tileID+'_'+prod_date
print('Tile & Date: ',name_out)

# the png will be saved under this name_out

print('----------------------------------------------------------------------------------------------')

##### READ RGB VALUES OUT OF JP2000 FILES

print('import RGB values and scale the array...')


# bands are read in the order B04 B03 B02 for RGB
ims = list( reversed( list( map( glymur.Jp2k, [ img_data + c for c in colors ]))))
scale = 8 # 8 is a good compromise between speed and detail


raw_bands = [i[::scale, ::scale] for i in ims]

rgb, nx, ny = np.shape(raw_bands)

print('----------------------------------------------------------------------------------------------')

##### CHOOSE METHOD TO PROCESS IMAGE DATA
##### adapthist, eq_hist, special or stretch
import image_processing_short as ips
hist_method = 'stretch'    
new_bands = np.ones((rgb+1,nx+2,ny+2))


#### IMAGE PROCESSING

# use two different methods of handling pixel values

bands_array = np.asarray(raw_bands)
bands = (ips.imageStretch(bands_array, 0, 4000))
bands = bands/255.1

print np.max(bands)

for i in range(3):
    placeholder = np.vstack([ np.ones(nx), bands[i] ])
    placeholder = np.vstack([ placeholder, np.ones(ny) ])
    placeholder = np.column_stack([ placeholder, np.ones(ny+2) ])
    placeholder = np.column_stack([ np.ones(ny+2), placeholder ])
    new_bands[i] = placeholder

arr1 = np.dstack([new_bands[0], new_bands[1], new_bands[2], new_bands[3]])

# second method

bands = (ips.image_percentile_clip(bands_array, 5, 95))
bands = bands/255.1

print np.max(bands)

for i in range(3):
    placeholder = np.vstack([ np.ones(nx), bands[i] ])
    placeholder = np.vstack([ placeholder, np.ones(ny) ])
    placeholder = np.column_stack([ placeholder, np.ones(ny+2) ])
    placeholder = np.column_stack([ np.ones(ny+2), placeholder ])
    new_bands[i] = placeholder

arr2 = np.dstack([new_bands[0], new_bands[1], new_bands[2], new_bands[3]])


# the bands matrix is of type 'list'


print('----------------------------------------------------------------------------------------------')

##### GET THE COORDINATES OF THE CORNERS

import xml.etree.ElementTree as et
import math

print('get lat/lon values of image corners...')

tree = et.parse(prefix+'MTD_MSIL1C.xml')
root = tree.getroot()
a = root.find('.//EXT_POS_LIST')
x = [float(i) for i in a.text.split()]     # convert values from string to float
del(x[-4:])                                # delete obsolete values at end of array
lat = x[0::2]
lon = x[1::2]
print(x)

print('----------------------------------------------------------------------------------------------')

##### SET UP THE COORDINATE SYSTEM OF THE ORIGINAL, TILTED IMAGE

# whereas lam & phi (small letters) denote the coordinates for each pixel in the original image

import numpy as np

for i in range(4):
    print(lat[i],lon[i])


print('set up coordinate system of tilted image...')

# the full-size image consists of 10'980 x 10'980 datapoints, i.e. pixel. due to memory
# restrictions, the number of pixels is reduced to 10'980 / x where x = 2^n.

n = np.shape(new_bands)[1]
m = np.shape(new_bands)[2]

phi_SWtoNW = np.linspace(lat[3], lat[0], num=n)
phi_SEtoNE = np.linspace(lat[2], lat[1], num=n)
lam_SWtoNW = np.linspace(lon[3], lon[0], num=m)
lam_SEtoNE = np.linspace(lon[2], lon[1], num=m)

lam = np.ones((n,m))
phi = np.ones((n,m))

a = 0

for i in range(n):
    a = a + 1
    phi[:][-a] = np.linspace(phi_SWtoNW[i], phi_SEtoNE[i], num=m)
    lam[:][-a] = np.linspace(lam_SWtoNW[i], lam_SEtoNE[i], num=n)


print('----------------------------------------------------------------------------------------------')

print('set up untilted coordinate system ...')


# set up the untilted coordinate system
M = n
N = m

Lam = np.ones((n,m))
Phi = np.ones((n,m))

#lamlam = np.linspace(np.min(lam), np.max(lam), M)         # try with lat and lon:
phiphi = np.linspace(np.min(phi), np.max(phi), N)          # lam = lon, phi = lat

a = 0

for i in range(n):
    a = a + 1
    Phi[:][-a] = phiphi[i]
    Lam[:][-a] = np.linspace(np.min(lam), np.max(lam), N)


print('----------------------------------------------------------------------------------------------')

print('zip Lam/Phi and lam/phi matrizes together for the KD Tree...')

goal_img = zip(Phi.ravel(), Lam.ravel())
orig_img = zip(phi.ravel(), lam.ravel())

print('----------------------------------------------------------------------------------------------')

print('create index matrix for the extraction of the RGB values of the unzipped matrix')
# create matrix with indizes of the original Phi/Lam & phi/lam matrizes:

indizes = np.zeros((N*M,2), dtype='i')

i = 0
j = 0
l = 0
running = True

while running:
    
    indizes[l] = [i, j]
    
    if j == n-1:
        i = i + 1
        j = -1
            
    j = j + 1
    l = l + 1
    
    if i == n:
        running = False
        
print('----------------------------------------------------------------------------------------------')

##### SET UP KD-TREE OF THE TILTED COORDINATE SYSTEM AND FIND NEAREST NEIGHBORS

from scipy import spatial

print('create KD Tree of lam/phi data pairs...')

import time

start = time.time()
tree = spatial.cKDTree(orig_img)
end = time.time()
print(end - start)

#print('----------------------------------------------------------------------------------------------')

print('iterating over the whole goal image to find nearest neighbors and assign RGB values to it...')
print

# two methods of image processing requires two resulting arrays
new_arr1 = np.ones((M,N,4))
new_arr2 = np.ones((M,N,4))
a = 0
b = 0
num = M*N                                 #number of iterations for statistics of periods
periods = np.empty((num,2))

start_tot = time.time()

for l in range(num):

    start_query = time.time()
    query = tree.query(goal_img[l])
    end_query = time.time()
    periods[l,0] = end_query - start_query

    start_rest = time.time()
    if all(arr1[indizes[query[1], 0], indizes[query[1], 1]][x] ==1. for x in range(4)):
        new_arr1[a][b] = 0
        new_arr2[a][b] = 0
    else:
        new_arr1[a][b] = arr1[indizes[query[1], 0], indizes[query[1], 1]]
        new_arr2[a][b] = arr2[indizes[query[1], 0], indizes[query[1], 1]]

    b = b + 1
    
    if l == ((a+1)*N)-1:
        a = a + 1
        b = 0

    end_rest = time.time()    
    periods[l,1] = end_rest - start_rest
        
end_tot = time.time()

# statistics

tot_time = end_tot - start_tot

print('times [s] to find nearest neighbor: mean of all queries = ', np.mean(periods[:,0]), 'total time = ', np.sum(periods[:,0]))
print('times [s] to assign RGB values: mean of all assignments =  ', np.mean(periods[:,1]), 'total time = ', np.sum(periods[:,1]))

print('total time [s] = ', end_tot - start_tot)
print


print('----------------------------------------------------------------------------------------------')

print('plotting section including saving the figure...')

# first time is for plotting the figure in the notebook if show_plot = True:

import matplotlib.pylab as plt

show_plot = False

if show_plot:

    plt.rcParams['figure.figsize'] = (n/50, m/50) 

    fig = plt.figure()
    ax = fig.add_axes([0, 0, 1, 1])
    plt.axis('off')
    plt.imshow(new_arr)
    plt.show()

# second time is for saving the picture as a png file:

plt.rcParams['figure.figsize'] = (n/50, n/50)

fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1])
plt.axis('off')
plt.imshow(new_arr1)

extent = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
plt.savefig('/home/eouser/Documents/Plots/'+name_out+'_1.png', transparent=True, bbox_inches=extent)

plt.rcParams['figure.figsize'] = (n/50, n/50)

fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1])
plt.axis('off')
plt.imshow(new_arr2)

extent = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
plt.savefig('/home/eouser/Documents/Plots/'+name_out+'_2.png', transparent=True, bbox_inches=extent)

print('----------------------------------------------------------------------------------------------')

get all the required names/strings of the product... 
('Tile & Date: ', 'T33XWH_20170402')
----------------------------------------------------------------------------------------------
import RGB values and scale the array...
----------------------------------------------------------------------------------------------
0.999607996864
0.999607996864
----------------------------------------------------------------------------------------------
get lat/lon values of image corners...
[79.26920705844542, 14.999037817529745, 79.22476283394975, 20.266794015488497, 78.24475761377488, 19.83217264795705, 78.28539372135954, 14.999117612618793]
----------------------------------------------------------------------------------------------
(79.26920705844542, 14.999037817529745)
(79.22476283394975, 20.266794015488497)
(78.24475761377488, 19.83217264795705)
(78.28539372135954, 14.999117612618793)
set up coordinate system of tilted image...
------------------------------------------------------------